# Языковые модели

Какое слово в последовательности вероятнее: 

Поезд прибыл на
* вокзал
* север

Какая последовательность вероятнее:
* Вокзал прибыл поезд на
* Поезд прибыл на вокзал

Языковая модель [language model, LM]  позволяет оценить вероятность следующего слова в последовательности  $P(w_n | w_1, \ldots, w_{n-1})$ и оценить вероятность всей последовательности слов $P(w_1, \ldots, w_n)$.

### Приложения:
* Задачи, в которых нужно обработать сложный и зашумленный вход: распознавание речи, распознавание сканированных и рукописных текстов;
* Исправление опечаток
* Машинный перевод
* Подсказка при наборе 

### Виды моделей:
* Счетные модели
    * цепи Маркова
* Нейронные модели, обычно реккурентные нейронные сети с LSTM/GRU
* seq2seq архитектуры

## Модель $n$-грам

Пусть $w_{1:n}=w_1,\ldots,w_m$ – последовательность слов.

Цепное правило:
$ P(w_{1:m}) = P(w_1) P(w_2 | w_1) P(w_3 | w_{1:2}) \ldots P(w_m | w_{1:m-1}) = \prod_{k=1}^{m} P(w_k | w_{1:k-1}) $

Но оценить $P(w_k | w_{1:k-1})$ не легче! 

Переходим к $n$-грамам: $P(w_{i+1} | w_{1:i}) \approx P(w_{i+1} | w_{i-n:i})  $ , то есть, учитываем $n-1$ предыдущее слово. 

Модель
* униграм: $P(w_k)$
* биграм: $P(w_k | w_{k-1})$
* триграм: $P(w_k | w_{k-1} w_{k-2})$


Т.е. используем Марковские допущения о длине запоминаемой цепочки.

* Вероятность следующего слова в последовательности: $ P(w_{i+1} | w_{1:i}) \approx P(w_{i-n:i}) $
* Вероятность всей последовательности слов: $P(w_{1:n}) = \prod_{k=1}^l P(w_k | w_{k-n+1: k-1}) $

### Качество модели  $n$-грам

Перплексия: насколько хорошо модель предсказывает выборку. Чем ниже значение перплексии, тем лучше.

$PP(\texttt{LM}) = 2 ^ {-\frac{1}{m} \log_2 \texttt{LM} (w_i | w_{1:i-1})}$

## Счетные языковые модели

### ММП оценки вероятностей
$ P_{MLE}(w_k | w_{k-n+1:k-1}) = \frac{\texttt{count}(w_{k-n+1:k-1} w_k )}{\texttt{count}(w_{k-n+1:k-1} )} $

В модели биграм:

$ P_{MLE}(w_k | w_{k-1}) = \frac{\texttt{count}(w_{k-1} w_k )}{\texttt{count}(w_{k-1} )} $

Возникает проблема нулевых вероятностей!

### Аддитивное сглаживание Лапласа

$ P(w_k | w_{k-1}) = \frac{\texttt{count}(w_{k-1} w_k ) + \alpha}{\texttt{count}(w_{k-1} ) + \alpha |V|} $

## Модели биграм в NLTK

In [1]:
import nltk
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, TimeDistributed, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout

In [2]:
names = [name.strip().lower() for name in 
         open('C:/Users/adwiz/Documents/Courses/datascience_netology/datasets/nlp/dinos.txt').readlines()]
print(names[:10])

['aachenosaurus', 'aardonyx', 'abdallahsaurus', 'abelisaurus', 'abrictosaurus', 'abrosaurus', 'abydosaurus', 'acanthopholis', 'achelousaurus', 'acheroraptor']


In [3]:
chars = [char for name in names for char in name]
freq = nltk.FreqDist(chars)
print(list(freq.keys()))

['a', 'c', 'h', 'e', 'n', 'o', 's', 'u', 'r', 'd', 'y', 'x', 'b', 'l', 'i', 't', 'p', 'v', 'm', 'g', 'f', 'j', 'k', 'w', 'z', 'q']


In [4]:
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(chars))
cfreq['a']

FreqDist({'u': 792, 'n': 354, 't': 213, 's': 187, 'l': 146, 'r': 131, 'c': 109, 'p': 96, 'm': 74, 'e': 48, ...})

In [5]:
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)
print('p(a a) = %1.4f' %cprob['a'].prob('a'))
print('p(a b) = %1.4f' %cprob['a'].prob('b'))
print('p(a u) = %1.4f' %cprob['a'].prob('u'))

p(a a) = 0.0105
p(a b) = 0.0129
p(a u) = 0.3185


In [6]:
from math import log
log(cprob['a'].prob('a')) + log(cprob['a'].prob('b')) + log(cprob['a'].prob('c'))

-12.041317008359863

In [7]:
l = sum([freq[char] for char in freq])
def unigram_prob(char):
    return freq[char] / l
print(f'p(a) = {unigram_prob("a"):1.4f}')
# print('p(a) = %1.4f' %unigram_prob('a'))

p(a) = 0.1354


In [8]:
[bi for bi in nltk.bigrams('aachenosaurus')]

[('a', 'a'),
 ('a', 'c'),
 ('c', 'h'),
 ('h', 'e'),
 ('e', 'n'),
 ('n', 'o'),
 ('o', 's'),
 ('s', 'a'),
 ('a', 'u'),
 ('u', 'r'),
 ('r', 'u'),
 ('u', 's')]

In [9]:
cprob["a"].generate()

'u'

#### Задание 1

1. Напишите функцию для оценки вероятности имени динозавра. 
2. Найдите наиболее вероятное имя динозавра из данного списка. 

In [10]:
def name_prob(name):
    p = unigram_prob(name[0])
    for i in range(len(name) - 1):
        p *= cprob[name[i]].prob(name[i+1])
    return p

In [11]:
name_prob(names[0])

2.0222358416238476e-10

In [12]:
names[0]

'aachenosaurus'

In [13]:
def generate_name(cprob, first_char, num_chars):
    name = ''
    name += first_char
    for i in range(num_chars):
        char = cprob[first_char].generate()
        name += char
        first_char = char
    return name

generate_name(cprob, 't', 9)

'tosajilino'

## Нейронные языковые модели

* Вход: $n$-грамы $w_{1:k}$
* $v(w_i)$ – эмбеддинг слова $w_i$, $v(w_i) \in \mathbb{R}^{d_{emb}}$, $d_{emb}$ – размерность эмбеддинга, $v(w) = E_{[w]}$
* $x = [v(w_1), v(w_2), \ldots , v(w_k)]$

$\widehat{y} = P(w_i | w_{1:k} ) = \texttt{LM}(w_{1:k}) = \texttt{softmax}(hW^2 +b^2)$

$h = g(xW^1+b^1)$

$w_i \in V$, $E \in \mathbb{R}^{|V|\times d_{emb}}, W^1 \in \mathbb{R}^{k \cdot d_{emb} \times d_{hid}}, b^1 \in \mathbb{R} ^ {d_{hid}}, W^2 \in \mathbb{R}^{d_{hid} \times |V|}, b^2 \in \mathbb{R} ^ {|V|}$

### Семплирование в нейронных языковых моделях 
### (Генерация текстов с помощью нейронных языковых моделей)

1. Задать начальную последовательность символов длины $k$ (/слов)
2. Предсказать распределение вероятностей слов с условием на $k$ предыдущих слов
3. 1. Выбрать слово с наибольшей вероятностью
3. 2. Выбрать слово по предсказаному распределению
4. Сдвинуть окно на одно слово и повторить 

#### Линейный поиск  (beam search)
Всегда помним $h$ наиболее вероятных гипотез:
1. Для генерации первого слова в последоватительности генерируем $h$ кандидатов, а не 1
2. Генерируем $h \times h$ кандидатов для второго слова и храним только $h$ наиболее вероятных


In [14]:
alphabet = list(set(chars))
print('total chars:', len(alphabet))

total chars: 26


In [17]:
maxlen = 5
step = 1
ngrams = []
next_chars = []
for name in names:
    for i in range(0, len(name) - maxlen, step):
        ngrams.append(' '.join([char for char in name[i: i + maxlen]]))
        next_chars.append(name[i + maxlen])
print('nb ngrams:', len(ngrams))
print(ngrams[0], next_chars[0])
print(ngrams[1], next_chars[1])

nb ngrams: 10701
a a c h e n
a c h e n o


In [18]:
tokenizer = Tokenizer(num_words=len(alphabet))
tokenizer.fit_on_texts(ngrams)

sequences = tokenizer.texts_to_sequences(ngrams)
X_train = pad_sequences(sequences, maxlen=maxlen)
sequences = tokenizer.texts_to_sequences(next_chars)
y_train = tokenizer.sequences_to_matrix(sequences)
X_train[0]

array([ 1,  1, 12, 11,  7])

In [19]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [21]:
char_index = tokenizer.word_index
index_char = {i: c for c, i in char_index.items()}

In [37]:
model = Sequential()
model.add(Embedding(len(alphabet), 50, input_length=maxlen))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(len(alphabet), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=2, verbose=0)

In [39]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) # temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.choice(range(len(alphabet)), p=preds)
    return probas

In [40]:
import random
import sys
import numpy as np

generated = ''
seed = 'aleks'
generated += seed
print('----- Generating with seed: "' + seed + '"')
print(generated)

for i in range(8):
    sequences = tokenizer.texts_to_sequences([' '.join([char for char in generated[-maxlen:]])])
    X_pred = pad_sequences(sequences, maxlen=maxlen)
    preds = model.predict(X_pred, verbose=0)[0]
    next_index = sample(preds)
    next_char = index_char[next_index]
    generated += next_char
    print(generated)

----- Generating with seed: "aleks"
aleks
aleksa
aleksan
aleksani
aleksanio
aleksanios
aleksaniosa
aleksaniosau
aleksaniosaur
